In [1]:
import json
import pandas as pd

# Bentso technologies

the uri is `http://rdf.bonsai.uno/activitytype/entsoe/` + the `Bentso technology` substituting the blackspaces by _ and / by `or`

e.g. 'http://rdf.bonsai.uno/activitytype/entsoe/Fossil_Brown_coalorLignite' for 'Fossil Brown coal/Lignite'

In [2]:
bentso_label_uri=pd.read_csv('../final_tables/tables/bentso_activity_types_uri.csv')
bentso_label_uri.head(1)

,Bentso technology,uri
0,Fossil Hard coal,http://rdf.bonsai.uno/activitytype/entsoe/Foss...


In [89]:
exiobase2bentso=pd.read_csv('../final_tables/tables/exiobase_to_bentso_activities.csv')
exiobase2bentso.head(1)

,Exiobase Activity name,Bentso technology
0,Cultivation of paddy rice,NaN


In [90]:
exiobase_labels=pd.read_csv('../raw/exiobase_v3_3_17_activities.csv')
exiobase_labels.head(1)

,Activity name,Activity code 1,Activity code 2
0,Cultivation of paddy rice,i01.a,A_PARI


In [91]:
df=pd.merge(exiobase2bentso.dropna(),
         exiobase_labels,
         left_on='Exiobase Activity name',
         right_on='Activity name')\
.set_index('Exiobase Activity name')[['Activity code 2','Bentso technology']]

In [106]:
ex2bentso_act=pd.merge(df,bentso_label_uri,on='Bentso technology')
ex2bentso_act.head(1)

,Activity code 2,Bentso technology,uri
0,A_POWC,Fossil Hard coal,http://rdf.bonsai.uno/activitytype/entsoe/Foss...


In [108]:

ex2bentso_act=ex2bentso_act.rename({'Activity code 2':'brdfat',
                                    'Bentso technology':'rdfs:label',
                                    'uri':'brdfatb'
                                   },
                                   axis=1)
#ex2bentso_act.head(1)

ex2bentso_act.brdfatb=ex2bentso_act.brdfatb.str.extract('http://rdf.bonsai.uno/activitytype/entsoe/(.*)')

ex2bentso_act.head(1)

,brdfat,rdfs:label,brdfatb
0,A_POWC,Fossil Hard coal,Fossil_Hard_coal


## turtle

In [145]:
f= open("../final_tables/tables_rdf_turtle/entsoe.ttl","w+")

f.write('@prefix bont: <http://ontology.bonsai.uno/core#> .\n')
f.write('@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n')
f.write('@prefix brdfatb: <http://rdf.bonsai.uno/activitytype/entsoe/> .\n')
f.write('@prefix brdfat: <http://rdf.bonsai.uno/activitytype/exiobase3_3_17/> .\n')
f.write('@prefix owl: <http://www.w3.org/2002/07/owl#> .\n')
f.write('@prefix dtype: <http://purl.org/dc/dcmitype/> .\n')
f.write('@prefix dc: <http://purl.org/dc/terms/> .\n')
f.write('@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n')
f.write('@prefix ns0: <http://purl.org/vocab/vann/> .\n')
f.write('@prefix cc: <http://creativecommons.org/ns#> .\n')
f.write('@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n')


f.write('\n')

f.write('<http://rdf.bonsai.uno/activitytype/entsoe/>\n')
f.write('  a dtype:Dataset ;\n')
f.write('  dc:title "Entsoe activity types"@en ;\n')
f.write('  dc:description "relation between entsoe electricity generators and exiobase"@en ;\n')
f.write('  foaf:homepage <http://rdf.bonsai.uno/activitytype/entsoe/documentation.html> ;\n')
f.write('  ns0:preferredNamespaceUri "http://rdf.bonsai.uno/activitytype/entsoe/#" ;\n')
f.write('  owl:versionInfo "Version 0.1 - 2019-03-25"@en ;\n')
f.write('  dc:modified "2019-03-25"^^xsd:date ;\n')
f.write('  dc:publisher "bonsai.uno" ;\n')
f.write('  dc:creator <http://bonsai.uno/foaf/bonsai.rdf#bonsai> ;\n')
f.write('  dc:contributor "Miguel F. Astudillo";\n')
f.write('  cc:license <http://creativecommons.org/licenses/by/3.0/> ;\n')
f.write('  rdfs:comment """First ever version 0.1 :Will change!"""@en .\n')

f.write('\n')

for r in ex2bentso_act.iterrows():  
    
    f.write('brdfatb:{} a bont:ActivityType;\n'.format(r[1]['brdfatb']))
    f.write('  owl:sameAs brdfat:{};\n'.format(r[1]['brdfat']))
    f.write('  rdfs:label:"{}".\n'.format(r[1]['rdfs:label']))
    f.write('\n')
    
f.close()